In [1]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
%run "txt_extractor.ipynb"
import config 


In [2]:
# this connects to the database

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=config.password,
  database="abstracts"
)

In [3]:
### this code is for extracting all data necessary for speechgraphs analysis from the database (Rejected Abstracts)) ###

mycursor = mydb.cursor()
mycursor.execute('SELECT a.Content, v.positivity, m.Publish_Date, m.Science\
                  FROM meta m\
                  JOIN valence v ON m.AbstractID = v.AbstractID\
                  JOIN abstract a ON m.AbstractID = a.AbstractID\
                  WHERE a.Rejected = 1 AND m.Science = "psy" OR a.Rejected = 1 AND m.Science = "phy" OR a.Rejected = 1 AND m.Science = "bio" OR a.Rejected = 1 AND m.Science = "hum"')

df = pd.DataFrame(mycursor.fetchall(), columns=['Content', 'Positivity', 'Publish_Date', 'Science'])

In [4]:
df.Science.unique()

array(['bio', 'psy', 'hum', 'phy'], dtype=object)

In [4]:
write_split_abstracts(df, "E:\\Speechgraphs_paper\\rejected_abstracts", thirds=True, combine_dates=True, sampling = False)

  0%|          | 0/11521 [00:00<?, ?it/s]

100%|██████████| 418/418 [00:00<00:00, 411843.80it/s]


In [5]:
### this code is for extracting all data necessary for speechgraphs analysis from the database ###

mycursor = mydb.cursor()
mycursor.execute('SELECT a.Content, v.positivity, m.Publish_Date, m.Science\
                  FROM meta m\
                  JOIN valence v ON m.AbstractID = v.AbstractID\
                  JOIN abstract a ON m.AbstractID = a.AbstractID\
                  WHERE a.Rejected = 0')

df = pd.DataFrame(mycursor.fetchall(), columns=['Content', 'Positivity', 'Publish_Date', 'Science'])
df = df.astype({'Science': 'category'})

In [7]:
write_split_abstracts(df, "E:\\Speechgraphs_paper", sample_size = 450, thirds=True, sampling = True)

100%|██████████| 509678/509678 [00:02<00:00, 230696.57it/s]


In [29]:
### extracting journals with highest pos value for different conditions ###

# all Sciences, at least 100 articles per journal
mycursor = mydb.cursor()
mycursor.execute('SELECT m.Journal, AVG(v.Positivity), COUNT(m.Journal)\
                  FROM meta m\
                  JOIN valence v ON m.AbstractID = v.AbstractID\
                  JOIN abstract a ON m.AbstractID = a.AbstractID\
                  WHERE a.Rejected = 0\
                  GROUP BY m.Journal\
                  HAVING COUNT(m.Journal) > 100\
                  ORDER BY AVG(v.Positivity) desc')

myresult = mycursor.fetchall()
plotdata = pd.DataFrame(myresult, columns=['Journal', 'Positivity', 'Count'])
plotdata.iloc[0:100].to_csv(r'E:\Journals_Study\100_most_pos_with_100_articles.csv', index = False)


# all Sciences, at least 1000 articles per journal
mycursor = mydb.cursor()
mycursor.execute('SELECT m.Journal, AVG(v.Positivity), COUNT(m.Journal)\
                  FROM meta m\
                  JOIN valence v ON m.AbstractID = v.AbstractID\
                  JOIN abstract a ON m.AbstractID = a.AbstractID\
                  WHERE a.Rejected = 0\
                  GROUP BY m.Journal\
                  HAVING COUNT(m.Journal) > 1000\
                  ORDER BY AVG(v.Positivity) desc')

myresult = mycursor.fetchall()
plotdata = pd.DataFrame(myresult, columns=['Journal', 'Positivity', 'Count'])
plotdata.iloc[0:100].to_csv(r'E:\Journals_Study\100_most_pos_with_1000_articles.csv', index = False)


# all Sciences, at least 5000 articles per journal
mycursor = mydb.cursor()
mycursor.execute('SELECT m.Journal, AVG(v.Positivity), COUNT(m.Journal)\
                  FROM meta m\
                  JOIN valence v ON m.AbstractID = v.AbstractID\
                  JOIN abstract a ON m.AbstractID = a.AbstractID\
                  WHERE a.Rejected = 0\
                  GROUP BY m.Journal\
                  HAVING COUNT(m.Journal) > 5000\
                  ORDER BY AVG(v.Positivity) desc')

myresult = mycursor.fetchall()
plotdata = pd.DataFrame(myresult, columns=['Journal', 'Positivity', 'Count'])
plotdata.iloc[0:100].to_csv(r'E:\Journals_Study\100_most_pos_with_5000_articles.csv', index = False)


# psychology, at least 100 articles per journal
mycursor = mydb.cursor()
mycursor.execute('SELECT m.Journal, AVG(v.Positivity), COUNT(m.Journal)\
                  FROM meta m\
                  JOIN valence v ON m.AbstractID = v.AbstractID\
                  JOIN abstract a ON m.AbstractID = a.AbstractID\
                  WHERE a.Rejected = 0 AND m.Science = "psy"\
                  GROUP BY m.Journal\
                  HAVING COUNT(m.Journal) > 100\
                  ORDER BY AVG(v.Positivity) desc')

myresult = mycursor.fetchall()
plotdata = pd.DataFrame(myresult, columns=['Journal', 'Positivity', 'Count'])
plotdata.iloc[0:100].to_csv(r'E:\Journals_Study\psy_100_most_pos_with_100_articles.csv', index = False)

In [4]:
print(myresult)

[('Patient education and counseling', 0.04339757881851159, 4130), ('BMC bioinformatics', 0.0419826674288328, 8308), ('Nutrients', 0.04016553077403019, 3081), ('Supportive care in cancer : official journal of the Multinational Association of Supportive Care in Cancer', 0.03977297835981219, 3079), ('Studies in health technology and informatics', 0.039296789037622536, 7443), ('Academic medicine : journal of the Association of American Medical Colleges', 0.03897986799647277, 3179), ('BMC health services research', 0.038916640668063746, 3191), ('Applied microbiology and biotechnology', 0.03861172652806395, 3070), ('IEEE transactions on medical imaging', 0.03762692098326103, 3514), ('International journal of environmental research and public health', 0.03754114155164862, 11424), ('The American journal of cardiology', 0.03721983386612051, 3489), ('Journal of clinical oncology : official journal of the American Society of Clinical Oncology', 0.03715931064918439, 3829), ('Sensors (Basel, Switze

In [53]:
# this was used for sampeling the abstracts for the eyetracking paradigm
# dont change the code below

mycursor = mydb.cursor()

mycursor.execute('SELECT r.rAbstract, m.Publish_date, v.positivity\
                  FROM raw_text r\
                  JOIN valence v ON r.AbstractID = v.AbstractID\
                  JOIN meta m ON r.AbstractID = m.AbstractID\
                  WHERE m.Science = "psy" AND m.Publish_date BETWEEN "2020-01-01" AND "2020-12-31"')

myresult = mycursor.fetchall()

df = pd.DataFrame(myresult, columns = ['Content', 'Publish_date', 'Positivity'])

df.Positivity.quantile(q = 0.25)

df.Positivity.quantile(q = 0.75)


df_low_pos = df[df.Positivity <= 0.013]
df_high_pos = df[df.Positivity > 0.051]

high_sample = df_high_pos.sample(24, random_state = 1)
low_sample = df_low_pos.sample(24, random_state = 1)

high_sample.Content.to_csv(r'E:\Abstract_MA_Study\high_sample.csv', index = False)
low_sample.Content.to_csv(r'E:\Abstract_MA_Study\low_sample.csv', index = False)